In [1]:
import os
import glob
import pandas as pd

# Merging metadata files
- Merging metadata files collected from four databases
- Output: a single df with the wanted fields: 'title', 'authors', 'venue', 'year', 'citationCount', 'fieldsOfStudy', 'abstract', 'doi'

In [49]:
# Science Direct

# Collected Files exist in the folders:
# raw_ScienceDirect____ ScienceDirect___________ SD Disinfo: 10 files
#                   |                  |_____ SD fake news: 10 files
#                   |                  |_____ SD Spam: 10 files
#                   |                  |_____ SD troll: 10 files
#                   |__ ScienceDirect______ metadata_ScDr_misinformation.csv
#                                      |__ metadata_ScDr_rumor.csv


def merge_SD_multi(root_p, wanted_fields, wanted_size):
    query_name = {
        'SD Disinfo': 'disinformation',
        'SD fake news': 'fake+news',
        'SD Spam' : 'spam',
        'SD troll': 'troll'
    }
    all_data = []
    for fold in os.listdir(root_p): 
        print(query_name[fold])
        data = []
        for file_p in glob.glob('%s\%s\*.csv'%(root_p, fold)):
#             print(file_p)
            try:
                with open(file_p, 'r') as f:
                    df = pd.read_csv(f)
            except:
                with open(file_p, 'r', encoding = 'utf-8') as f:
                    df = pd.read_csv(f)        
            data.append(df)
            dfs = pd.concat(data)
            dfs['query'] = query_name[fold]
            
            if wanted_size is not None:
                wanted_size = wanted_size
            else:
                wanted_size = len(dfs)
#             print('df.shape', dfs.shape)        

            all_data.append(dfs[wanted_fields][:wanted_size])

    all_dfs = pd.concat(all_data)
    print('all_dfs.shape', all_dfs.shape)
    return all_dfs


def merge_SD_single(root_p, wanted_fields, wanted_size):
    data = []
    for file_p in glob.glob('%s\*.csv'%(root_p)):
        query = file_p.split('\\')[-1].split('_')[-1].split('.')[0]
        print(query)
        try:
            with open(file_p, 'r') as f:
                df = pd.read_csv(f)
        except:
            with open(file_p, 'r', encoding = 'utf-8') as f:
                df = pd.read_csv(f)
        df['query'] = query
        
        if wanted_size is not None:
                wanted_size = wanted_size
        else:
            wanted_size = len(df)
        data.append(df[wanted_fields][:wanted_size])
    dfs = pd.concat(data)
    print('dfs.shape', dfs.shape)
    return dfs


if __name__ == "__main__":
    wanted_fields = ['Item type', 'Authors', 'Title', 'Journal', 'Publication year',
       'Volume', 'Issue', 'Pages', 'Date published', 'ISSN', 'URLs', 'DOI',
       'Abstract', 'Keywords', 'Notes', 'query']
    root = r"C:\Users\hn0139\OneDrive - UNT System\A_PhD_PATH\PROJECTS\Misinformation\Misinformation_literature_review\metadata"
    out_p = root + '\merged_ScienceDirect'
        
    sd_multi_f = merge_SD_multi(
        root + "\\raw_ScienceDirect\\ScienceDirect", wanted_fields, wanted_size = None) 
    sd_single_f = merge_SD_single(
        root + "\\raw_ScienceDirect\\Science_direct", wanted_fields, wanted_size = None)
    concat_SD = pd.concat([sd_multi_f, sd_single_f])
    print('the SD concatenated data has %d examples, and %d columns.'%(concat_SD.shape[0], concat_SD.shape[1]))
    print(concat_SD.columns)

    try:
        os.makedirs(out_p, exist_ok = True)
    except OSError as error:
        print('Directory cannot be created!')
    with open(out_p + '\ScienceDirect.csv', 'w', encoding = 'utf-8', newline = '') as f:
        concat_SD.to_csv(f)





disinformation
fake+news
spam
troll
all_dfs.shape (4100, 16)
misinformation
rumor
dfs.shape (2119, 16)
the SD concatenated data has 6219 examples, and 16 columns.
Index(['Item type', 'Authors', 'Title', 'Journal', 'Publication year',
       'Volume', 'Issue', 'Pages', 'Date published', 'ISSN', 'URLs', 'DOI',
       'Abstract', 'Keywords', 'Notes', 'query'],
      dtype='object')


In [69]:
def merge_cross_DBs(root_p, wanted_fields, wanted_size=1000):
    fold_names = {
        'scopus': 'Scopus csv',
        'semantic_scholar': "Semantic_scholar",
        'wos': "WoS_Metadata",
        'science_direct': 'merged_ScienceDirect'
    }

    all_data = []
    for k, v in fold_names.items():
        print('\n_____%s_____' % k)
        fold_p = root_p + "\\" + v
        file_ps = [f for f in glob.glob(fold_p + '\*.csv')]
        if k == 'science_direct':
            if len(file_ps) == 1:
                with open(file_ps[0], 'r', encoding='utf-8') as f:
                    df = pd.read_csv(f)
            else:
                print('more than one file in the directory')
            print(df.columns)
            df.columns = ['Unnamed: 0', 'Item type', 'authors', 'title', 'venue', 'year', 'Volume', 'Issue', 'Pages',
                          'Date published', 
                          'ISSN', 'URLs', 'doi', 'abstract', 'Keywords', 'Notes', 'query']
            df['database'] = k
            new_cols = list(set(wanted_fields).difference(list(df.columns)))
            for col in new_cols:
                df[col] = ''
            df = df[wanted_fields]
            all_data.append(df)
        else:
            for path in file_ps:
                try:
                    with open(path, 'r') as f:
                        df = pd.read_csv(f)
                except:
                    with open(path, 'r', encoding='utf-8') as f:
                        df = pd.read_csv(f)
                if wanted_size < len(df):
                    wanted_size = 1000
                else:
                    wanted_size = len(df)
                print('wanted_size: %d'%wanted_size)
                
                if k == 'scopus':
                    df.columns = ['Item type', 'authors', 'title', 'venue', 'year', 'Volume',
                                  'Issue', 'Pages', 'Date published', 'URLs', 'doi', 'Notes', 'citationCount']

                    query = path.split('\\')[-1].split(' ')[-1].split('.')[0]
                elif k == 'wos':
                    df.columns = ['Publication Type', 'authors', 'Book Authors', 'Book Editors',
                                  'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
                                  'Group Authors', 'title', 'Source Title', 'Book Series Title',
                                  'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
                                  'Conference Date', 'Conference Location', 'Conference Sponsor',
                                  'Conference Host', 'Author Keywords', 'Keywords Plus', 'abstract',
                                  'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
                                  'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
                                  'Funding Text', 'Cited References', 'Cited Reference Count',
                                  'citationCount, WoS Core', 'Times Cited, All Databases',
                                  '180 Day Usage Count', 'Since 2013 Usage Count', 'venue',
                                  'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
                                  'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
                                  'year', 'Volume', 'Issue', 'Part Number', 'Supplement',
                                  'Special Issue', 'Meeting Abstract', 'Start Page', 'End Page',
                                  'Article Number', 'doi', 'DOI Link', 'Book DOI', 'Early Access Date',
                                  'Number of Pages', 'WoS Categories', 'Web of Science Index',
                                  'fieldsOfStudy', 'IDS Number', 'Pubmed Id', 'Open Access Designations',
                                  'Highly Cited Status', 'Hot Paper Status', 'Date of Export',
                                  'UT (Unique WOS ID)', 'Web of Science Record']
                    query = path.split('\\')[-1].split('_')[-1].split('.')[0]

                elif k == 'semantic_scholar':
                    query = path.split('\\')[-1].split('_')[-1].split('.')[0]
                else:
                    pass
                print('%s 11111 has %d instances.' % (k, df.shape[0]))
                df = df[:wanted_size]
                df['query'] = query

                df['database'] = k
                new_cols = list(set(wanted_fields).difference(list(df.columns)))
                for col in new_cols:
                    df[col] = ''
                df = df[wanted_fields]
                all_data.append(df)
                print('%s 2222 has %d instances.' % (k, df.shape[0]))

    all_dfs = pd.concat(all_data)
    print('\nDone! Merged df has %s examples, and %s fields' % (str(all_dfs.shape[0]), str(all_dfs.shape[1])))
    return all_dfs


if __name__ == "__main__":
    root_p = r"C:\Users\hn0139\OneDrive - UNT System\A_PhD_PATH\PROJECTS\Misinformation\Misinformation_literature_review\metadata"
    wanted_fields = ['title', 'authors', 'venue', 'year', 'citationCount', 'fieldsOfStudy', 'abstract', 'doi', 'query',
                     'database']
    merged_all_data = merge_cross_DBs(root_p, wanted_fields)

    out_p = root_p + '\merged_all_data'
    try:
        os.makedirs(out_p, exist_ok=True)
    except OSError as error:
        print('Directory cannot be created!')
    with open(out_p + '\merged_all_data.csv', 'w', encoding='utf-8', newline='') as f:
        merged_all_data.to_csv(f)


_____scopus_____
wanted_size: 1000
scopus 11111 has 2000 instances.
scopus 2222 has 1000 instances.
wanted_size: 1000
scopus 11111 has 2000 instances.
scopus 2222 has 1000 instances.
wanted_size: 1000
scopus 11111 has 2000 instances.
scopus 2222 has 1000 instances.
wanted_size: 1000
scopus 11111 has 2000 instances.
scopus 2222 has 1000 instances.
wanted_size: 1000
scopus 11111 has 2000 instances.
scopus 2222 has 1000 instances.
wanted_size: 897
scopus 11111 has 897 instances.
scopus 2222 has 897 instances.

_____semantic_scholar_____
wanted_size: 1000
semantic_scholar 11111 has 990 instances.
semantic_scholar 2222 has 990 instances.
wanted_size: 990
semantic_scholar 11111 has 990 instances.
semantic_scholar 2222 has 990 instances.
wanted_size: 990
semantic_scholar 11111 has 990 instances.
semantic_scholar 2222 has 990 instances.
wanted_size: 990
semantic_scholar 11111 has 990 instances.
semantic_scholar 2222 has 990 instances.
wanted_size: 990
semantic_scholar 11111 has 990 instances.

## Data cleaning
- Stardardize datatypes
- Normalize(lowercase, etc)
- Deduplicate
- write to a cleaned csv file


In [4]:
data_p = r"C:\Users\huyen\OneDrive - UNT System\A_PhD_PATH\PROJECTS\Misinformation\Misinformation_literature_review\metadata\merged_all_data\merged_all_data.csv"
with open(data_p, 'r', encoding='utf-8', newline='') as f:
    all_df = pd.read_csv(f)
all_df.head(3)

,Unnamed: 0,title,authors,venue,year,citationCount,fieldsOfStudy,abstract,doi,query,database
0,0,WHO competency framework for health authoritie...,"Rubinelli S,Purnat TD,Wihelm E,Traicoff D,Nama...",Human Resources for Health,2022,NaN,NaN,NaN,10.1186/s12960-022-00733-0,disinformation,scopus
1,1,A cross-sectional study of factors associated ...,"Yeager S,Abramovitz D,Harvey-Vera AY,Vera CF,A...",BMC Public Health,2022,NaN,NaN,NaN,10.1186/s12889-022-13273-y,disinformation,scopus
2,2,Mapping state-sponsored information operations...,"Uyheng J,Cruickshank IJ,Carley KM",EPJ Data Science,2022,NaN,NaN,NaN,10.1140/epjds/s13688-022-00338-6,disinformation,scopus


In [5]:
df_copy = all_df.copy()
df_copy 

,Unnamed: 0,title,authors,venue,year,citationCount,fieldsOfStudy,abstract,doi,query,database
0,0,WHO competency framework for health authoritie...,"Rubinelli S,Purnat TD,Wihelm E,Traicoff D,Nama...",Human Resources for Health,2022,NaN,NaN,NaN,10.1186/s12960-022-00733-0,disinformation,scopus
1,1,A cross-sectional study of factors associated ...,"Yeager S,Abramovitz D,Harvey-Vera AY,Vera CF,A...",BMC Public Health,2022,NaN,NaN,NaN,10.1186/s12889-022-13273-y,disinformation,scopus
2,2,Mapping state-sponsored information operations...,"Uyheng J,Cruickshank IJ,Carley KM",EPJ Data Science,2022,NaN,NaN,NaN,10.1140/epjds/s13688-022-00338-6,disinformation,scopus
3,3,A digital media literacy intervention for olde...,"Moore RC,Hancock JT",Scientific Reports,2022,NaN,NaN,NaN,10.1038/s41598-022-08437-0,disinformation,scopus
4,4,Lateral reading and monetary incentives to spo...,"Panizza F,Ronzani P,Martini C,Mattavelli S,Mor...",Scientific Reports,2022,NaN,NaN,NaN,10.1038/s41598-022-09168-y,disinformation,scopus
...,...,...,...,...,...,...,...,...,...,...,...
24051,6214,Effective strategies for responding to rumors ...,"Paek HJ,Hove T",Public Relations Review,2019,NaN,NaN,This experimental study explores how governmen...,10.1016/j.pubrev.2019.02.006,rumor,science_direct
24052,6215,Factors affecting individual online rumor shar...,"Luo P,Wang C,Guo F,Luo L",Computers in Human Behavior,2021,NaN,NaN,"With the outbreak of COVID-19, online sharing ...",10.1016/j.chb.2021.106968,rumor,science_direct
24053,6216,Stability and Hopf bifurcation analysis of mul...,"Wang J,Jiang H,Hu C,Yu Z,Li J","Chaos, Solitons & Fractals",2021,NaN,NaN,The stability and hopf bifurcation of the mult...,10.1016/j.chaos.2021.111464,rumor,science_direct
24054,6217,Soft rumor control in social networks: Modelin...,"Askarizadeh M,Tork Ladani B",Engineering Applications of Artificial Intelli...,2021,NaN,NaN,"Nowadays, social networks become ubiquitous pl...",10.1016/j.engappai.2021.104198,rumor,science_direct


In [7]:
df_copy.isnull().sum()

Unnamed: 0           0
title                0
authors            232
venue             3370
year               358
citationCount    17977
fieldsOfStudy    12321
abstract          7893
doi               7149
query                0
database             0
dtype: int64

In [8]:
df_copy.dtypes

Unnamed: 0        int64
title            object
authors          object
venue            object
year             object
citationCount    object
fieldsOfStudy    object
abstract         object
doi              object
query            object
database         object
dtype: object

In [13]:
def lowercase(x):
    return x.lower()
fields_to_dedup = ['title', 'authors', 'venue', 'doi']
for field in fields_to_dedup:
    print(field)
    df_copy[field] = df_copy[field].apply(lambda x: lowercase(x) if type(x)==str else x)
df_copy.head()

title
authors
venue
doi


,Unnamed: 0,title,authors,venue,year,citationCount,fieldsOfStudy,abstract,doi,query,database
0,0,who competency framework for health authoritie...,"rubinelli s,purnat td,wihelm e,traicoff d,nama...",human resources for health,2022,NaN,NaN,NaN,10.1186/s12960-022-00733-0,disinformation,scopus
1,1,a cross-sectional study of factors associated ...,"yeager s,abramovitz d,harvey-vera ay,vera cf,a...",bmc public health,2022,NaN,NaN,NaN,10.1186/s12889-022-13273-y,disinformation,scopus
2,2,mapping state-sponsored information operations...,"uyheng j,cruickshank ij,carley km",epj data science,2022,NaN,NaN,NaN,10.1140/epjds/s13688-022-00338-6,disinformation,scopus
3,3,a digital media literacy intervention for olde...,"moore rc,hancock jt",scientific reports,2022,NaN,NaN,NaN,10.1038/s41598-022-08437-0,disinformation,scopus
4,4,lateral reading and monetary incentives to spo...,"panizza f,ronzani p,martini c,mattavelli s,mor...",scientific reports,2022,NaN,NaN,NaN,10.1038/s41598-022-09168-y,disinformation,scopus


In [29]:
# Observe the number of duplicates.
print("number of duplicates using 'title',  'authors', 'venue', 'year': ", df_copy[df_copy.duplicated(subset = ['title',  'authors', 'venue','year'])].shape[0])
print("number of duplicates using 'title',  'authors', 'venue': ", df_copy[df_copy.duplicated(subset = ['title',  'authors', 'venue'])].shape[0])
print("number of duplicates using 'title',  'authors', 'year': ", df_copy[df_copy.duplicated(subset = ['title',  'authors', 'year'])].shape[0])
print("number of duplicates using 'title',  'year': ", df_copy[df_copy.duplicated(subset = ['title',  'year'])].shape[0])
print("number of duplicates using 'title', 'doi': ", df_copy[df_copy.duplicated(subset = ['title', 'doi'])].shape[0])
print("number of duplicates using 'title': ", df_copy[df_copy.duplicated(subset = ['title'])].shape[0])
print("number of duplicates using 'doi': ", df_copy[df_copy['doi'].str.len()>3].duplicated(subset = ['doi']).shape[0])


number of duplicates using 'title',  'authors', 'venue', 'year':  5413
number of duplicates using 'title',  'authors', 'venue':  5590
number of duplicates using 'title',  'authors', 'year':  5491
number of duplicates using 'title',  'year':  6284
number of duplicates using 'title', 'doi':  7264
number of duplicates using 'title':  8452
number of duplicates using 'doi':  16888


In [37]:
len(df_copy[df_copy['doi'].str.len()>3].drop_duplicates(subset = ['doi']))

9949

In [26]:
# deduplicate
df_deduplicated = df_copy[df_copy['doi'].str.len()>3].drop_duplicates(subset = ['title',  'year'])
len(df_deduplicated)

11631